## Setting up database connection

Importing required python libraries

In [41]:
import py2neo
from nesta.core.luigihacks.misctools import get_config
from nesta.core.orms.orm_utils import graph_session
import igraph as ig
import pandas as pd

Establish connection to the Neo4j database

In [3]:
conf = get_config('neo4j.config', 'neo4j')
gkwargs = dict(host=conf['host'], secure=True,
                auth=(conf['user'], conf['password']))

## Retrieving nodes from Neo4j

Create a graph object which will be used for our queries

In [16]:
with graph_session(**gkwargs) as tx:
    graph = tx.graph

Set the type of the node that should be retrieved. Available types are: "Project", "Organisation", "Publication", "Topic", "Report", "Datasets", "Software", "Proposal_Call". Simple change the word "Organisation" below to the required node type and re-run the cell.

In [42]:
node_type = "Organisation"

In [43]:
node_list = list(graph.nodes.match(node_type))
node_table = pd.DataFrame(node_list)
#Get first 5 nodes of the specified type
node_table.head(5)

,betw,country_code,country_name,id,name
0,0.000747,GB,United Kingdom,-99679701,BAE SYSTEMS (OPERATIONS) LIMITED
1,0.724385,VN,Vietnam,-99552481,Hanoi University of Technology
2,0.733103,GR,Greece,-99525116,COSMOTE KINITES TILEPIKOINONIES AE
3,0.947206,DK,Denmark,-99524889,CONFEDERATION OF DANISH INDUSTRIES
4,0.465352,CH,Switzerland,-99449792,Swiss Air Ambulance LTD


## Sorting the results

To sort the table according to a column, simply make the variable "sort_column" equal to the column name (e.g. "betw")

In [44]:
sort_column = 'betw'

This gets top 10 nodes sorted in the descending order according to column "betw" (betweenness centrality)

In [45]:
sorted_table = node_table.sort_values(by=[sort_column], ascending=False)
sorted_table.head(10)

,betw,country_code,country_name,id,name
8769,0.999999,DE,Germany,922470777,RATH GMBH
23798,0.999999,IL,Israel,953674610,SINGER INSTRUMENTS & CONTROL LTD
30460,0.999979,DK,Denmark,972429172,CITOXLAB SCANTOX AS
50867,0.999968,GB,United Kingdom,999948557,ANDOR TECHNOLOGY LIMITED
7457,0.999919,US,United States,918469818,"TOTAL E&P Research and Technology USA, LLC"
8432,0.999900,CO,Colombia,920903742,CORPORACION UNIVERSIDAD DE LA COSTA CUC
9748,0.999878,CN,China,925236829,NINGBO UNIVERSITY
35763,0.999870,DE,Germany,990692041,CROSSGATE AG
20644,0.999844,ES,Spain,951819388,GALLETAS GULLON SA
37564,0.999804,PE,Peru,994990014,Investigaciones Medicas en Salud


## Filtering the results

To filter the results table according to a column, simply make the variable "filter_column" equal to the column name (e.g. "country_code") and set the value of filter through the variable "filter_value".

In [46]:
filter_column = "country_code"
filter_value = "DE"

The example below shows first 10 organisations in Germany.

In [47]:
node_table.loc[node_table[filter_column] == filter_value]
node_table.head(10)

,betw,country_code,country_name,id,name
0,0.000747,GB,United Kingdom,-99679701,BAE SYSTEMS (OPERATIONS) LIMITED
1,0.724385,VN,Vietnam,-99552481,Hanoi University of Technology
2,0.733103,GR,Greece,-99525116,COSMOTE KINITES TILEPIKOINONIES AE
3,0.947206,DK,Denmark,-99524889,CONFEDERATION OF DANISH INDUSTRIES
4,0.465352,CH,Switzerland,-99449792,Swiss Air Ambulance LTD
5,0.178480,DE,Germany,-98900625,VDI/VDE INNOVATION + TECHNIK GMBH
6,0.626649,PL,Poland,-98701641,SZCZECINECKA LOKALNA ORGANIZACJA TURYSTYCZNA
7,0.068822,DE,Germany,-98374997,UNIVERSITAETSKLINIKUM AACHEN
8,0.866878,GR,Greece,-97632515,INTRACOM SA TELECOM SOLUTIONS
9,0.050070,PL,Poland,-96918953,POLITECHNIKA KOSZALINSKA


The example below shows top10 organisations in Germany sorted by betweenness centrality

In [48]:
sorted_table.loc[sorted_table[filter_column] == filter_value]
sorted_table.head(10)

,betw,country_code,country_name,id,name
8769,0.999999,DE,Germany,922470777,RATH GMBH
23798,0.999999,IL,Israel,953674610,SINGER INSTRUMENTS & CONTROL LTD
30460,0.999979,DK,Denmark,972429172,CITOXLAB SCANTOX AS
50867,0.999968,GB,United Kingdom,999948557,ANDOR TECHNOLOGY LIMITED
7457,0.999919,US,United States,918469818,"TOTAL E&P Research and Technology USA, LLC"
8432,0.999900,CO,Colombia,920903742,CORPORACION UNIVERSIDAD DE LA COSTA CUC
9748,0.999878,CN,China,925236829,NINGBO UNIVERSITY
35763,0.999870,DE,Germany,990692041,CROSSGATE AG
20644,0.999844,ES,Spain,951819388,GALLETAS GULLON SA
37564,0.999804,PE,Peru,994990014,Investigaciones Medicas en Salud


## Working with individual nodes

Find node by ID

In [ ]:
found_node = graph.nodes.get(323063)

List all attributes of a node

In [89]:
print(found_node.keys())

dict_keys(['website', 'funded_under', 'total_cost', 'acronym', 'betw', 'ec_contribution', 'title', 'objective', 'project_description', 'framework', 'start_date_code', 'end_date_code', 'funding_scheme', 'rcn', 'status'])


Get attribute "acronym" of the reference node

In [90]:
print(found_node['acronym'])

i3DPost


Find nodes that are connected (i.e. have a relationship) to the node of reference

In [94]:
#find linked nodes where the reference node is start node
connected_nodes = list()
for rel in graph.match((None, found_node)):
    connected_nodes.append(rel.nodes)

In [92]:
#find linked nodes where the reference node is end node
connected_nodes = list()
for rel in graph.match((found_node, None )):
    connected_nodes.append(rel.nodes)

In [93]:
#find all nodes that have a HAS_PROJECT relationship to the reference node
connected_nodes = list()
for rel in graph.match((None, found_node), r_type="HAS_PROJECT"):
    connected_nodes.append(rel.nodes)

In [96]:
connected_nodes_table = pd.DataFrame(connected_nodes)
connected_nodes_table.head()

,0,1
0,"{'betw': 0.45848474349594937, 'title': 'FP7-IC...","{'website': 'http://www.i3dpost.eu', 'funded_u..."
1,"{'betw': 0.13743845474192695, 'title': 'Intell...","{'website': 'http://www.i3dpost.eu', 'funded_u..."
2,"{'betw': 0.8886491203588955, 'project_rcn': 85...","{'website': 'http://www.i3dpost.eu', 'funded_u..."
3,"{'betw': 0.5459254756812018, 'project_rcn': 85...","{'website': 'http://www.i3dpost.eu', 'funded_u..."
4,"{'betw': 0.7404681112068626, 'name': 'ETHNIKO ...","{'website': 'http://www.i3dpost.eu', 'funded_u..."


In [61]:
#filter nodes by property value
found_nodes = pd.DataFrame(list(graph.nodes.match("Project", ec_contribution__gt=1000000, status__not="CLOSED") ) )
found_nodes.head()

,acronym,betw,ec_contribution,end_date_code,framework,funded_under,funding_scheme,objective,project_description,rcn,start_date_code,status,title,total_cost,website
0,GENESYS,0.815392,1079397,2010-04-12T00:00:00,FP7,"[Specific Programme ""Cooperation"": Information...",CSA - Coordination and support action,"So far, several EU funded projects and other i...",\nICT for Environmental Management and Energy ...,87319,2008-09-01T00:00:00,TERMINATED,Good practices for European developers of adva...,1615300,
1,IRMA,0.401757,2481395,2009-12-31T00:00:00,FP7,"[Specific Programme ""Cooperation"": Information...",CP - Collaborative project (generic),Disaster risk reduction policies and instituti...,\nICT for Environmental Management and Energy ...,87828,2008-06-01T00:00:00,TERMINATED,Integrated Risk Management for Africa,3533633,
2,INSPIRE,0.399926,4979995,2014-01-31T00:00:00,FP7,"[Specific programme ""People"" implementing the ...","MC-COFUND - Co-funding of Regional, National a...",The objective of this programme is to refine a...,,90290,2009-02-01T00:00:00,TERMINATED,IRCSET International Mobility Fellowships in S...,12449988,
3,NOVEL TOOLS IN PD,0.271529,1495400,2014-11-30T00:00:00,FP7,"[Specific programme: ""Ideas"" implementing the ...",ERC-SG - ERC Starting Grant,To understand the molecular basis of any biolo...,,92856,2009-12-01T00:00:00,TERMINATED,Novel tools for real time monitoring and quant...,1495400,
4,RDCVF,0.077114,2623333,2013-02-28T00:00:00,FP7,"[Specific Programme ""Cooperation"": Health]",CP-FP - Small or medium-scale focused research...,The discovery of RdCVF (Rod-derived Cone Viabi...,,94464,2010-03-01T00:00:00,TERMINATED,Rod-derived Cone Viability Factor,3934701,http://www.rdcvf.eu/
